# Visual secret sharing

## Tạo ảnh

B1 : Tạo ra 1 tấm ảnh img chứa mã xác thực tọa độ (0, img)

B2 : gán $x_0$ = $img_0$

B3 : xác định phương trình $y = w_0*x_0 + w_1*x_1 + ... + img$

B4 : chọn threshold là n => random generate $ w = [n - 1]$ ma trận

B5 : auto generate n ảnh dựa trên phương trình y

## Ghép ảnh
B1 : thu thập n tấm ảnh

B2 : chọn threshold là n => random generate $ w = [n]$ ma trận

B3 : trả về trọng số cuối 

In [1]:
from PIL import Image
import numpy as np
import random

In [168]:
class equationSystem:
    def __init__(self, sampleList, x_root):
        self.sampleList = sampleList
        self.matrix = list()
        self.x_root = x_root
    
    def solve(self):
        self.matrix = self.equationize()
        print(self.matrix)
        return self.solveMatrix()

    def equationize(self):
        num_equation = len(self.sampleList)
        matrix = list()

        for i in range(len(self.sampleList)):
            tempEquation = list()

            for j in range(num_equation):
                tempEquation.append((i + 5)**(num_equation - j - 1))
                
            matrix.append(tempEquation)
        
        return np.array(matrix)

    def solveMatrix(self):
        matrix = np.array(self.matrix)
        sampleList = list(self.sampleList)

        #Step 2 : calculate A^-1
        matrix_inv = np.linalg.inv(matrix)
        
        #Step 3 : x = A^-1 * B
        weight = list()
        for i in range(len(matrix_inv)):
            temp = np.zeros(shape=(500,500))
            for j in range(len(matrix_inv[i])):
                temp += matrix_inv[i][j]*sampleList[j]
            weight.append(temp)
        
        return weight[-1]
            

In [177]:
class VisualSecretSharing:
    def __init__(self, img=None, n_generator=3):
        self.img = img
        self.n_generator = n_generator
    
    def fusionImage(self, samplelist):
        self.sampleList = samplelist
        for i in range(len(self.sampleList)):
            self.sampleList[i] = np.array(self.sampleList[i], dtype='float64')
        
        self.sampleList = self.reverseMin_Max_Scaler(self.sampleList)
        predict = equationSystem(self.sampleList, self.x_root).solve()

        return Image.fromarray(predict)
        
    def createImage(self, n):
        if self.img == None:
            return None
        
        self.autoGenerateWeight()
        result = list()

        for i in range(n):
            x = i + 5
            print(x)
            temp = np.zeros(shape=(500,500))
            for j in range(self.n_generator):
                temp += self.weight[j]*(x**(self.n_generator - j - 1))
            result.append(temp)

        return self.Min_Max_Scaler(result)

    def autoGenerateSampleImg(self):
        sampleList = list()

        for i in range(self.n_generator - 1):
            temp = np.random.randint(0,255, size=(500,500), dtype='int32')
            sampleList.append(temp)
        
        return sampleList
    
    def autoGenerateWeight(self):        
        self.img = self.img.resize((500,500))
        self.img = self.img.convert('L')
        self.matrix = np.array(self.img)
        self.x_root = self.matrix[0][0]

        self.weight = self.autoGenerateSampleImg()
        self.weight.append(self.matrix)
    
    def Min_Max_Scaler(self, sampleList):
        result = list()
        self.min = 0
        self.max = 255
        for i in sampleList:
            if self.min > np.min(i):
                self.min = np.min(i)
            if self.max < np.max(i): 
                self.max = np.max(i) 
        
        for i in sampleList:
            result.append(i / (self.max - self.min) * 255)
        
        return result

    def reverseMin_Max_Scaler(self, sampleList):
        result = list()
        for i in sampleList:
            result.append(i / 255 * self.max)
        
        return result

In [182]:
img = Image.open('D:/Project/Python/Algorithm_Design/template/image/TDT_logo.png')
vss = VisualSecretSharing(img, n_generator=3)
temp = vss.createImage(3)

inputS = random.sample(temp, 3)

for i in temp:
        print(i,end='\n\n\n')
        (Image.fromarray(i)).show()

# print(temp[-1])
# print()
vss.fusionImage(temp).show()
# for i in temp:
#     print(i)
    # new_img = Image.fromarray(i)
    # new_img.show()

5
6
7
[[ 82.24670212  99.24200566  42.18005387 ... 111.74632226 102.13032668
   16.32605843]
 [ 34.07866565  82.86311209  68.24538987 ...  26.76980454  82.00013813
   98.88977139]
 [101.79570412 114.74031356  95.80772153 ...  56.70971752  98.83693625
   99.06588853]
 ...
 [ 97.09337661  15.70964846  67.1006285  ... 102.50017266  64.89916431
  128.03715726]
 [ 62.96187582  92.69044824 112.82063678 ... 116.76566061  81.63029215
   97.21665861]
 [102.99330064  70.97520547 106.90310104 ...  74.84978244  45.96657228
   82.33476069]]


[[112.48601423 140.91131984  53.62766766 ... 158.54064507 142.56682091
   22.43732302]
 [ 45.70239657 116.29014435  93.78237447 ...  36.70281097 115.56806409
  138.28717453]
 [142.07369293 159.72062988 134.88811382 ...  79.92195594 136.61406174
  138.32239796]
 ...
 [135.9095932   18.95020374  93.50058706 ... 144.18709856  87.86483873
  179.05829132]
 [ 85.29352856 126.7867256  160.33703985 ... 163.20774915 113.43704676
  137.63554113]
 [140.9817667   97.48083

In [60]:
a = np.random.randint(1,10,size=(2,2))
b = np.random.randint(1,10,size=(2,2))
c = np.random.randint(1,10,size=(2,2))

def f(a,b,c,x):
    return a*x**2 + b*x + c

y_template = list()
matrix = list()
for i in range(3):
    temp = random.randint(2,10)
    for j in range(3):
        matrix.append(temp**(2 - j))
    y_template.append(f(a,b,c,temp))

matrix = np.array(matrix)
matrix = matrix.reshape(3,3)
print(matrix)
arr_inv = np.linalg.inv(matrix)
print(arr_inv)

result = list()
for i in range(3):
    row = arr_inv[i]
    rs = 0
    for j in range(3):
        rs += row[j]*y_template[j]
    result.append(rs)

[[16  4  1]
 [ 4  2  1]
 [25  5  1]]
[[-0.5         0.16666667  0.33333333]
 [ 3.5        -1.5        -2.        ]
 [-5.          3.33333333  2.66666667]]


In [64]:
print(c)
print(result[2])

[[8 8]
 [1 5]]
[[8. 8.]
 [1. 5.]]
